In [277]:
import time
import pandas as pd
import os
import requests
import urllib
from bs4 import BeautifulSoup
import re
import numpy as np

In [222]:
movie_meta = pd.read_csv('./movie_meta.csv',header=0,encoding='CP949')
#movie_meta.head(5)
num = 194
print(movie_meta['Keyword'][num],movie_meta['open_day_30'][num],movie_meta['open_day_60'][num])
movie_meta.shape

어벤져스 엔드게임 2019-03-25 2019-06-23


(641, 4)

In [199]:
page = 1
num = 0 ~ 641
start_dt = movie_meta['open_day_30'][num]
end_dt = movie_meta['open_day_60'][num]
orgin_movie_nm = movie_meta['Keyword'][num]
movie_nm = urllib.parse.quote(orgin_movie_nm, encoding='CP949')

In [224]:
movie_meta.Keyword

0              언니
1           데드풀 2
2      레인보우 나의 사랑
3              레토
4       미션 이스탄불 5
          ...    
636       와일드 라이프
637    고흐 영원의 문에서
638      백두 번째 구름
639    천문 하늘에 묻는다
640          미드웨이
Name: Keyword, Length: 641, dtype: object

In [242]:
rslt_list_final = []
for num in range(movie_meta.shape[0]):
    start_dt = movie_meta['open_day_30'][num]
    end_dt = movie_meta['open_day_60'][num]
    orgin_movie_nm = movie_meta['Keyword'][num]
    movie_nm = urllib.parse.quote(orgin_movie_nm, encoding='CP949')
    print(list(movie_meta.Keyword).index(orgin_movie_nm), orgin_movie_nm)
    
    find_url = 'https://cafe.naver.com/ArticleSearchList.nhn?search.clubid=10000052&search.media=0&search.searchdate='+str(start_dt)+str(end_dt)+'&search.exact=&search.include=&userDisplay=50&search.exclude=&search.option=0&search.sortBy=date&search.searchBy=1&search.includeAll=&search.query='+movie_nm+'&search.viewtype=title&search.page='+str(page)

    resp = requests.get(find_url)
    content = resp.content.decode('ms949')
    iframe = BeautifulSoup(content, "html.parser").find("div", {"class":"skin-1080 fl"})
    trs = iframe.find_all('tr', {'class': None, 'id': None})
    iframe2 = BeautifulSoup(content, "html.parser").find("div", {"class":"prev-next"})
    page_cnt = iframe2.find_all('a')
    pg_cnt = re.sub('\n|&lt|&gt|;|\s{2,}', '', re.sub('<[^>]*>', '', str(page_cnt)))
    # print(page_cnt)
    pg_cnt1 = pg_cnt.rstrip(']')
    pg_cnt2 = pg_cnt1.lstrip('[')
    if pg_cnt2 != '':
        max_pg_cnt = max(pg_cnt2)
        # print(max_pg_cnt)

        rslt_list = []
        for pg_num in range(1, int(max_pg_cnt)+1):
#             print(pg_num)
            find_url_final = 'https://cafe.naver.com/ArticleSearchList.nhn?search.clubid=10000052&search.media=0&search.searchdate='+str(start_dt)+str(end_dt)+'&search.exact=&search.include=&userDisplay=50&search.exclude=&search.option=0&search.sortBy=date&search.searchBy=1&search.includeAll=&search.query='+movie_nm+'&search.viewtype=title&search.page='+str(pg_num)
#             print(find_url_final)
            resp = requests.get(find_url_final)
            content = resp.content.decode('ms949')
            iframe = BeautifulSoup(content, "html.parser").find("div", {"class":"skin-1080 fl"})
            trs = iframe.find_all('tr', {'class': None, 'id': None})

            m = [i   for i in range(0, len(trs), 2)]
            n = [i+1 for i in range(0, len(trs), 2)] # 불필요

            rslt = pd.DataFrame()
            for m, n in zip(m, n):
                s = trs[m].find('td', {'class': 'td_article'}).find('a', {'class':'article'}) if (trs[m] and trs[m].find('td', {'class': 'td_article'})) is not None else None
                t = trs[m].find('a', {'class': 'm-tcol-c'}) if trs[m] is not None else None
                u = trs[m].find('em', {'class': None}) if trs[m] is not None else None
                v = trs[m].find('td', {'class': 'td_date'}) if trs[m] is not None else None
                w = trs[m].find('td', {'class': 'td_view'}) if trs[m] is not None else None


                title = re.sub(',', '', re.sub('\n|&lt|&gt|;|\s{2,}', '', re.sub('<[^>]*>', '', str(s))))
                author = re.sub('\n|&lt|&gt|;|\s{2,}', '', re.sub('<[^>]*>', '', str(t)))
                comments = re.sub('\n|&lt|&gt|;|\s{2,}', '', re.sub('<[^>]*>', '', str(u)))
                date = re.sub('\.', '',re.sub('\n|&lt|&gt|;|\s{2,}', '', re.sub('<[^>]*>', '', str(v))))
                view = re.sub(',', '', re.sub('\n|&lt|&gt|;|\s{2,}', '', re.sub('<[^>]*>', '', str(w))))

        #         print("\n====================================")
        #         print(title)  # title
        #         print(author) # author
        #         print(comments)      # comments
        #         print(date)      # date
        #         print(view)      # view

                new_row = {'movie_keyword': orgin_movie_nm, 'title': title, 'author': author, 'comments': comments, 'date': date, 'view': view}
                rslt = rslt.append(new_row, ignore_index = True)
                rslt = rslt[['movie_keyword', 'title' ,'author' ,'comments','date','view']]

            rslt_list.append(rslt)
        rslt_final = pd.concat(rslt_list)
        rslt_list_final.append(rslt_final)
rslt_final_final = pd.concat(rslt_list_final)

0 언니
1 데드풀 2
2 레인보우 나의 사랑
3 레토
4 미션 이스탄불 5
5 주먹왕 랄프 2
6 아이스브레이커
7 그린북
8 내안의 그놈
9 말모이
10 극장판 공룡메카드 타이니소어의 섬
11 돈겟아웃
12 다스 크레이지 하트
13 리지
14 빌리어네어 보이즈클럽
15 사무라이 검신 주베에
16 신 배틀로얄 극악 소년원
17 신 배틀로얄 극악 소년원 2
18 완벽한 그녀의 비밀
19 요코하마 갱스터
20 아스트로 우주전쟁
21 구스 베이비
22 그대 이름은 장미
23 미래의 미라이
24 언더독
25 왕이 될 아이
26 쿠르스크
27 글래스
28 더 서치
29 디 엔드
30 메이트
31 언싱커블
32 우행록 어리석은 자의 기록
33 일일시호일
34 창간호
35 극한직업
36 두번째 여름
37 몬스터파크
38 크리미널 게임 보석 사기단
39 행맨
40 가버나움
41 데들리 스토커
42 베스와 베라
43 아이 킬 자이언츠
44 얼굴들
45 파이널리스트
46 하트스트링스
47 트라이아스기 월드 잃어버린 세계
48 고검기담 소명신검의 부활
49 드래곤 길들이기 3
50 뺑반
51 이월
52 jk닌자걸스
53 극장판 헬로카봇 옴파로스 섬의 비밀
54 우리가족 라멘샵
55 시인 할매
56 알리타 배틀앤젤
57 툼 인베이더
58 레고 무비 2
59 사랑은 비가 갠 뒤처럼
60 아이스
61 일진 3
62 챔피언스
63 콜드워
64 기묘한 가족
65 신밧드와 마법 양탄자
66 증인
67 닥터 킬러 패밀리
68 도쿄의 밤하늘은 항상 가장 짙은 블루
69 드래곤볼 슈퍼 브로리
70 러브라이브 선샤인 더 스쿨 아이돌 무비 오버 더 레인보우
71 메리 포핀스 리턴즈
72 메리 퀸 오브 스코틀랜드
73 명탐정 코난 전율의 악보
74 타임 워프 워
75 해피 데스데이 2 유
76 해피니스 로드
77 험악한 꿈
78 5 솔저스
79 치트 더 행맨
80 포 핸즈
81 모험왕 블링키
82 사바하
83 신데렐라 마법 반지의 비밀
84 콜드 체이싱
85 CCTV 은밀한 시선
86 더 페이버릿 

In [241]:
rslt_list_final

[   movie_keyword                                        title  \
 0             언니                        [스포X][언니] 또 하나의 졸작 탄생   
 1             언니                      [티켓인증]영화 언니-앤디unic1004님   
 2             언니                      앤디님옘댕으로..언니보러...감사합니다^^   
 3             언니                    [완료]언니 cgv1매 예매--투유진님께 완료   
 4             언니                  [언니] 분노폭팔한 언니의 처절한 복수극(스포X)   
 5             언니                 스윙키즈 마약왕 언니 PMC 더 벙커 아쉽네요...   
 6             언니                        [티켓인증]와우님께서 나눔해주신 언니~   
 7             언니                 [언니] 동생을 지키기위해 언니가 나서다(스폰있음)   
 8             언니                      [언니] 왕십리씨지브이 1시영화에 무대인사   
 9             언니                        [티켓인증]토리파님 양도로 온 [언니]   
 10            언니                   [언니] 화끈한 액션 어디갔나요 ㅠㅠㅠ(스포X)   
 11            언니          [티켓인증]쉬린님 나눔으로 언니보러왔어요 화끈한 액션 기대 ㅋㅋ   
 12            언니                [언니] 여성 단독 액션영화로 손꼽힐 영화 (약스포)   
 13            언니                         영화 [ 언니 ] 관람후기 (스포O)   
 14       

In [245]:
rslt_final_final.head(10)

,movie_keyword,title,author,comments,date,view
0,언니,[스포X][언니] 또 하나의 졸작 탄생,케이론 chiron94,None,20190117,26
1,언니,[티켓인증]영화 언니-앤디unic1004님,하루랑 jis1128,2,20190113,20
2,언니,앤디님옘댕으로..언니보러...감사합니다^^,맘보요 jht0911,3,20190113,24
3,언니,[완료]언니 cgv1매 예매--투유진님께 완료,쓔쓔 csjini0109,9,20190111,148
4,언니,[언니] 분노폭팔한 언니의 처절한 복수극(스포X),쏭 ssoj0827,None,20190110,61
5,언니,스윙키즈 마약왕 언니 PMC 더 벙커 아쉽네요...,오늘도반가워gigon384,2,20190109,127
6,언니,[티켓인증]와우님께서 나눔해주신 언니~,뮬이무 khg 1209,None,20190105,13
7,언니,[언니] 동생을 지키기위해 언니가 나서다(스폰있음),파란만장 huyongman,None,20190105,2293
8,언니,[언니] 왕십리씨지브이 1시영화에 무대인사,파란만장 huyongman,4,20190105,160
9,언니,[티켓인증]토리파님 양도로 온 [언니],최고중최고 y5498383,None,20190104,17


In [251]:
len(rslt_final_final.movie_keyword.unique())

458

In [252]:
rslt_final_final.to_csv('./nagongca.csv',sep="|",index=False, mode='w',encoding='utf-8-sig',header=False)

In [281]:
np.nan_to_num(rslt_final_final.to_numpy)

<bound method DataFrame.to_numpy of    movie_keyword                                      title          author  \
0             언니                      [스포X][언니] 또 하나의 졸작 탄생    케이론 chiron94   
1             언니                    [티켓인증]영화 언니-앤디unic1004님     하루랑 jis1128   
2             언니                    앤디님옘댕으로..언니보러...감사합니다^^     맘보요 jht0911   
3             언니                  [완료]언니 cgv1매 예매--투유진님께 완료   쓔쓔 csjini0109   
4             언니                [언니] 분노폭팔한 언니의 처절한 복수극(스포X)      쏭 ssoj0827   
..           ...                                        ...             ...   
44          미드웨이                      미드웨이 12/23 롯시월타 19:30    제이 yohan0521   
45          미드웨이  [완료][미드웨이] 23일 롯데월타 19:30분 일정입니다.(☞수류화개님)    깔끔신사 0416jsh   
46          미드웨이                  미드웨이 블록버스터 시사회 초대 (12/23)  네영카movie020202   
47          미드웨이                    미드웨이 2차 예고편 보셨나요? 대박이에요           단헐적간식   
48          미드웨이             미드웨이 포스터 설마 이게 메인이 아니길 바래 봅니다.  네영카movie020202   

   comments    

In [276]:
rslt_final_final

,movie_keyword,title,author,comments,date,view
0,언니,[스포X][언니] 또 하나의 졸작 탄생,케이론 chiron94,None,20190117,26
1,언니,[티켓인증]영화 언니-앤디unic1004님,하루랑 jis1128,2,20190113,20
2,언니,앤디님옘댕으로..언니보러...감사합니다^^,맘보요 jht0911,3,20190113,24
3,언니,[완료]언니 cgv1매 예매--투유진님께 완료,쓔쓔 csjini0109,9,20190111,148
4,언니,[언니] 분노폭팔한 언니의 처절한 복수극(스포X),쏭 ssoj0827,None,20190110,61
...,...,...,...,...,...,...
44,미드웨이,미드웨이 12/23 롯시월타 19:30,제이 yohan0521,6,20191219,93
45,미드웨이,[완료][미드웨이] 23일 롯데월타 19:30분 일정입니다.(☞수류화개님),깔끔신사 0416jsh,7,20191219,93
46,미드웨이,미드웨이 블록버스터 시사회 초대 (12/23),네영카movie020202,154,20191209,2556
47,미드웨이,미드웨이 2차 예고편 보셨나요? 대박이에요,단헐적간식,8,20191204,46


In [273]:
?pd.DataFrame.astype

In [ ]:
rslt_final_final.groupby(['movie_keyword']).aggr()